In [ ]:
from music21 import *
import pickle
import glob


def parse_mid():
    notes = []
    duration = []

    for midfile in glob.glob('midi_source/*.mid'):

        mid = converter.parse(midfile)
        
        notes_to_parse = None

        parts = instrument.partitionByInstrument(mid)

        if parts:  # file has instrument parts
            notes_to_parse = parts.parts[0].recurse()

        else:  # file has notes in a flat structure
            notes_to_parse = mid.flat.notes
            print('teste')

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                a = str(element.pitch)
                duration.append(str(element.duration.quarterLength))
                a += ("|")
                a += str(element.duration.quarterLength)
                notes.append(a)


            elif isinstance(element, chord.Chord):
                chord_pitches=[]
                

                for p in element.pitches:
                    chord_pitches.append(p.midi)

                
                chord_pitches = str(chord_pitches).strip('[]')
                
                
               
                chord_pitches += ("|")
                
                chord_duration = str(element.duration.quarterLength)
                
                
                chord_pitches += chord_duration
                chord_pitches += ("|chord")

            
                notes.append(chord_pitches)
                with open('data/notes', 'wb') as path:
                    pickle.dump(notes, path)
        

        print(notes)
parse_mid()

['C#2|0.25', 'G#2|0.25', '49, 44|0.25|chord', 'E3|0.25', '56, 37|0.25|chord', 'C#3|0.25', '52, 44|0.25|chord', 'G#3|0.25', '61, 37|0.25|chord', 'E3|0.25', '56, 44|0.25|chord', 'C#4|0.25', '64, 37|0.25|chord', 'G#3|0.25', '61, 44|0.25|chord', 'E4|0.25', '68, 37|0.25|chord', 'C#4|0.25', '64, 44|0.25|chord', 'G#4|0.25', '73, 37|0.25|chord', 'E4|0.25', '68, 44|0.25|chord', 'C#5|0.25', '76, 37|0.25|chord', 'G#4|0.25', '73, 44|0.25|chord', 'E5|0.25', '68, 76, 80, 73, 49, 37|0.25|chord', '68, 73, 80, 76, 44|0.25|chord', 'C2|0.25', 'G#2|0.25', '48, 44|0.25|chord', 'E-3|0.25', '56, 36|0.25|chord', 'C3|0.25', '51, 44|0.25|chord', 'G#3|0.25', '60, 36|0.25|chord', 'E-3|0.25', '56, 44|0.25|chord', 'C4|0.25', '63, 36|0.25|chord', 'G#3|0.25', '60, 44|0.25|chord', 'E-4|0.25', '68, 36|0.25|chord', 'C4|0.25', '63, 44|0.25|chord', 'G#4|0.25', '72, 36|0.25|chord', 'E-4|0.25', '68, 44|0.25|chord', 'C5|0.25', '75, 36|0.25|chord', 'G#4|0.25', '72, 44|0.25|chord', 'E-5|0.25', '68, 72, 75, 80, 48, 36|0.25|chor

['C#2|0.25', 'G#2|0.25', '49, 44|0.25|chord', 'E3|0.25', '56, 37|0.25|chord', 'C#3|0.25', '52, 44|0.25|chord', 'G#3|0.25', '61, 37|0.25|chord', 'E3|0.25', '56, 44|0.25|chord', 'C#4|0.25', '64, 37|0.25|chord', 'G#3|0.25', '61, 44|0.25|chord', 'E4|0.25', '68, 37|0.25|chord', 'C#4|0.25', '64, 44|0.25|chord', 'G#4|0.25', '73, 37|0.25|chord', 'E4|0.25', '68, 44|0.25|chord', 'C#5|0.25', '76, 37|0.25|chord', 'G#4|0.25', '73, 44|0.25|chord', 'E5|0.25', '68, 76, 80, 73, 49, 37|0.25|chord', '68, 73, 80, 76, 44|0.25|chord', 'C2|0.25', 'G#2|0.25', '48, 44|0.25|chord', 'E-3|0.25', '56, 36|0.25|chord', 'C3|0.25', '51, 44|0.25|chord', 'G#3|0.25', '60, 36|0.25|chord', 'E-3|0.25', '56, 44|0.25|chord', 'C4|0.25', '63, 36|0.25|chord', 'G#3|0.25', '60, 44|0.25|chord', 'E-4|0.25', '68, 36|0.25|chord', 'C4|0.25', '63, 44|0.25|chord', 'G#4|0.25', '72, 36|0.25|chord', 'E-4|0.25', '68, 44|0.25|chord', 'C5|0.25', '75, 36|0.25|chord', 'G#4|0.25', '72, 44|0.25|chord', 'E-5|0.25', '68, 72, 75, 80, 48, 36|0.25|chor

['C#2|0.25', 'G#2|0.25', '49, 44|0.25|chord', 'E3|0.25', '56, 37|0.25|chord', 'C#3|0.25', '52, 44|0.25|chord', 'G#3|0.25', '61, 37|0.25|chord', 'E3|0.25', '56, 44|0.25|chord', 'C#4|0.25', '64, 37|0.25|chord', 'G#3|0.25', '61, 44|0.25|chord', 'E4|0.25', '68, 37|0.25|chord', 'C#4|0.25', '64, 44|0.25|chord', 'G#4|0.25', '73, 37|0.25|chord', 'E4|0.25', '68, 44|0.25|chord', 'C#5|0.25', '76, 37|0.25|chord', 'G#4|0.25', '73, 44|0.25|chord', 'E5|0.25', '68, 76, 80, 73, 49, 37|0.25|chord', '68, 73, 80, 76, 44|0.25|chord', 'C2|0.25', 'G#2|0.25', '48, 44|0.25|chord', 'E-3|0.25', '56, 36|0.25|chord', 'C3|0.25', '51, 44|0.25|chord', 'G#3|0.25', '60, 36|0.25|chord', 'E-3|0.25', '56, 44|0.25|chord', 'C4|0.25', '63, 36|0.25|chord', 'G#3|0.25', '60, 44|0.25|chord', 'E-4|0.25', '68, 36|0.25|chord', 'C4|0.25', '63, 44|0.25|chord', 'G#4|0.25', '72, 36|0.25|chord', 'E-4|0.25', '68, 44|0.25|chord', 'C5|0.25', '75, 36|0.25|chord', 'G#4|0.25', '72, 44|0.25|chord', 'E-5|0.25', '68, 72, 75, 80, 48, 36|0.25|chor